In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.5 MB/s eta 0:00:00


In [34]:
!pip install -qU bs4 tiktoken openai langchain pinecone-client[grpc]
     

In [28]:
import requests

res = requests.get("https://www.acciona.com/es/soluciones/")
res
     

<Response [200]>

In [29]:
from bs4 import BeautifulSoup
import urllib.parse
import html
import re

domain = "https://www.acciona.com/es/soluciones/"
domain_full = domain+"es/soluciones/"

soup = BeautifulSoup(res.text, 'html.parser')

# Find all links to local pages on the website
local_links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    if href.startswith(domain) or href.startswith('./') \
        or href.startswith('/') or href.startswith('modules') \
        or href.startswith('use_cases'):
        local_links.append(urllib.parse.urljoin(domain_full, href))

# Find the main content using CSS selectors
main_content = soup.select('body main')[0]

# Extract the HTML code of the main content
main_content_html = str(main_content)

# Extract the plaintext of the main content
main_content_text = main_content.get_text()

# Remove all HTML tags
main_content_text = re.sub(r'<[^>]+>', '', main_content_text)

# Remove extra white space
main_content_text = ' '.join(main_content_text.split())

# Replace HTML entities with their corresponding characters
main_content_text = html.unescape(main_content_text)

print(main_content_text)

PaísesClientesContactoEnglishÁrea de ClientesNuestro propósitoSolucionesProyectosAccionistas e inversoresActualidadNuestro propósitoHistoriaSostenibilidad Nuestra estrategiaObjetivos de Desarrollo SostenibleEmergencia climáticaDesarrollo socialÍndices de sostenibilidadAlianzas para el Desarrollo SosteniblePortada de SostenibilidadInnovación Centros de innovaciónInnovación abierta y colaborativaDigital hubProyectos de innovación Portada de InnovaciónNuestra fundación: acciona.orgIniciativas que apoyamosPor qué trabajar con nosotros Ofertas de empleoPrograma de graduados y becasDiversidad, equidad e inclusiónSeguridad, salud y bienestarFormaciónPortada de Por qué trabajar con nosotrosPortada de Nuestro propósitoSolucionesEnergíaTransporteAguaSocialesCiudadesInmobiliariaFinancierasPortada de SolucionesProyectosAccionistas e inversoresInformación financiera Informe anual integradoCuentas anualesInformes de resultados y presentacionesPeríodo medio de pago a proveedoresRatingPortada de Infor

In [30]:
def scrape(url: str):
    res = requests.get(url)
    if res.status_code != 200:
        print(f"{res.status_code} for '{url}'")
        return None
    soup = BeautifulSoup(res.text, 'html.parser')

    # Find all links to local pages on the website
    local_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith(domain) or href.startswith('./') \
            or href.startswith('/') or href.startswith('modules') \
            or href.startswith('use_cases'):
            local_links.append(urllib.parse.urljoin(domain_full, href))

    # Find the main content using CSS selectors
    main_content = soup.select('body main')[0]

    # Extract the HTML code of the main content
    main_content_html = str(main_content)

    # Extract the plaintext of the main content
    main_content_text = main_content.get_text()

    # Remove all HTML tags
    main_content_text = re.sub(r'<[^>]+>', '', main_content_text)

    # Remove extra white space
    main_content_text = ' '.join(main_content_text.split())

    # Replace HTML entities with their corresponding characters
    main_content_text = html.unescape(main_content_text)

    # return as json
    return {
        "url": url,
        "text": main_content_text
    }, local_links

In [31]:
links = ["https://www.acciona.com/es/soluciones/"]
scraped = set()
data = []

while True:
    if len(links) == 0:
        print("Complete")
        break
    url = links[0]
    print(url)
    res = scrape(url)
    scraped.add(url)
    if res is not None:
        page_content, local_links = res
        data.append(page_content)
        # add new links to links list
        links.extend(local_links)
        # remove duplicates
        links = list(set(links))
    # remove links already scraped
    links = [link for link in links if link not in scraped]

https://www.acciona.com/es/soluciones/
https://www.acciona.com/es/soluciones/energia/areas-actividad/solar-fotovoltaica/
https://www.acciona.com/es/soluciones/transporte/areas-actividad/puentes/
502 for 'https://www.acciona.com/es/soluciones/transporte/areas-actividad/puentes/'
https://www.acciona.com/es/nuestro-proposito/innovacion/
https://www.acciona.com/es/desarrollo-sostenible/
https://www.acciona.com/es/soluciones/ciudades/
https://www.acciona.com/es/nuestro-proposito/trabaja-con-nosotros/ofertas-empleo/
https://www.acciona.com/es/soluciones/sociales/areas-actividad/salud/
https://www.acciona.com/es/proyectos/resultados/?solution=Sociales&area=Salud
https://www.acciona.com/es/
https://www.acciona.com/es/nuestro-proposito/trabaja-con-nosotros/formacion/
https://www.acciona.com/es/nuestro-proposito/sostenibilidad/alianzas-desarrollo-sostenible/
https://www.acciona.com/es/soluciones/sociales/proyectos/
https://www.acciona.com/es/soluciones/ciudades/sostenibilidad/
https://www.accion

In [35]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)
     

In [36]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [13]:
#Process the data into more chunks 



In [37]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record['url']
    } for i in range(len(texts))])

  0%|          | 0/457 [00:00<?, ?it/s]

In [ ]:
# Initialize Embedding Model

In [38]:
import boto3
import json
import os

# especifica el nombre del archivo de configuración y del bucket de S3
bucket_name = 'opstack-bucket'
config_file_name = 'config.json'

# crea un cliente de S3 y descarga el archivo de configuración
s3 = boto3.client('s3')
s3.download_file(bucket_name, config_file_name, '/tmp/config.json')

# carga la variable de entorno desde el archivo de configuración
with open('/tmp/config.json') as f:
    config = json.load(f)
    os.environ.update(config)

# comprueba si la variable de entorno se ha cargado correctamente
#print(os.getenv('PINECONE_API_KEY'))
#print(os.getenv('PINECONE_ENVIRONMENT'))


In [6]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.4 MB/s eta 0:00:00


In [39]:
import openai

# get API key from top-right dropdown on OpenAI website
openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

openai.Engine.list()  # check we have authenticated

<OpenAIObject list at 0x7f7c60376810> JSON: {
  "data": [
    {
      "created": null,
      "id": "babbage",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "davinci",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-davinci-edit-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-code-search-code",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-similarity-babbage-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-davinci-edit-001",
      "object": "engine",
      

In [ ]:
models = openai.Model.list()
embeddings = [model.id for model in models["data"] if "text-" in model.id]
print(embeddings)
#escogemos el modelo de embeddings que se ajuste a nuestras necesidades

In [40]:
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In [41]:
res.keys()


dict_keys(['object', 'data', 'model', 'usage'])

In [42]:

len(res['data'])

2

In [43]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])


(1536, 1536)

## Aplicando Embeddings a los documentos de acciona

Ahora que hemos extraído los documentos de acciona, aplicaremos la misma lógica de embeddings para representarlos en forma vectorial. Sin embargo, antes de hacerlo, necesitamos crear un lugar donde almacenar estos embeddings.

### Inicializando el índice

Para almacenar los embeddings y habilitar una búsqueda eficiente de vectores a través de ellos, utilizaremos Pinecone. Podemos obtener una clave de API gratuita y podemos usarlo desde aws marketplace con el correo asociado a nuestro usuario de iam y luego inicializar nuestra conexión a Pinecone y crear un nuevo índice en el siguiente bloque de código:



In [14]:
!pip install pinecone-client[grpc]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 2.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 3.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.5 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.0/218.0 kB 3.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.0/218.0 kB 3.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 2.8 M

In [44]:
import pinecone
from tqdm.autonotebook import tqdm
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

pinecone.init(api_key=api_key, enviroment=env)
pinecone.whoami()

WhoAmIResponse(username='f088b0c', user_label='default', projectname='5a663c9')

In [21]:
index_name = 'gpt-4-acciona-webdocs'


In [24]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)


In [45]:
index.describe_index_stats()
#view index stats

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Podemos ver que el índice está actualmente vacío con un `total_vector_count` de 0. Podemos comenzar a poblarlo con embeddings pre-entrenados de OpenAI `text-embedding-ada-002` de la siguiente manera:





In [46]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)
     

  0%|          | 0/39 [00:00<?, ?it/s]

Ahora hemos agregado todos nuestros webdocdocs de acciona al índice. Con eso, podemos pasar a la recuperación y luego a la generación de respuestas usando GPT-4.

Recuperación:
Para buscar en nuestros documentos, primero necesitamos crear un vector de consulta xq. Usando xq, recuperaremos los fragmentos más relevantes de los documentos de Acciona, así:

In [47]:
query = "que servicios ofrece acciona?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [48]:
res


{'matches': [{'id': '76daf8bd-1845-4552-b233-9da7fb96c398',
              'metadata': {'chunk': 4.0,
                           'text': 'de limpieza y plataformas innovadoras para '
                                   'la gestión de la logística interna. '
                                   'Asimismo, ACCIONA dispone de asistencia y '
                                   'transporte sanitario urgente y no urgente '
                                   'y dota a sus profesionales de planes '
                                   'específicos de formación continua, gestión '
                                   'de calidad del servicio, contingencias, '
                                   'emergencias y autoprotección para el '
                                   'personal. ACCIONA lleva a cabo la gestión '
                                   'integral del uso de la energía que abarca '
                                   'tanto la demanda como la oferta, con un '
                                   'm

Retrieval Augmented Generation:

Actualmente, se accede a GPT-4 a través del punto final ChatCompletions de OpenAI. Para agregar la información que recuperamos al modelo, necesitamos pasarla a nuestras solicitudes de usuario junto con nuestra consulta original. Podemos hacerlo de la siguiente manera:

In [49]:

# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query
     

In [50]:
print(augmented_query)


de limpieza y plataformas innovadoras para la gestión de la logística interna. Asimismo, ACCIONA dispone de asistencia y transporte sanitario urgente y no urgente y dota a sus profesionales de planes específicos de formación continua, gestión de calidad del servicio, contingencias, emergencias y autoprotección para el personal. ACCIONA lleva a cabo la gestión integral del uso de la energía que abarca tanto la demanda como la oferta, con un modelo de gestión basado en dos rasgos diferenciales: la sostenibilidad y la digitalización. Diseñamos soluciones para reducir y optimizar el consumo energético de los hospitales, consiguiendo importantes ahorros económicos y reduciendo el impacto ambiental y las emisiones de CO₂.Hospital Infanta Sofía: ganador del premio BREEAM 2020 por su eﬁciencia energética y gestión hídricaHospital de Vigo: ganador del premio “Future Health Project 2012” al mejor proyecto europeo de saludProyectos destacados de salud Ver todos los proyectos de salud Suscríbete a

aunque parece que se ha equivocado vemos la cadena "EnergíaTransporteAguaSocialesCiudadesInmobiliariaFinancieras" hay que tenmer en cuenta que esto solo es un scrape de una pagina web, puede contener algun error aun así ha dado una respuesta satisfactorias los servcicios son relacionados con energia, transporte, agua, sociales, inmobiliaria, financiera, ademas vemos servicios para sus empleados, lo cual podriamos decir que con algo de limpieza en el cuerpo de datos los resultados son prometedores:

ahora vamos a hacer un poco de prompt engineer para afilar a nuestro asistente:

In [51]:
# Mensaje de sistema para 'iniciar' el modelo
primer = """You are a Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information cannot be found in the provided
data, you should truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query},
        {"role": "assistant", "content": "Please respond in the language the user writes in."}
    ]
)


mostramos esto de forma bonita: 
To display this response nicely, we will display it in markdown.

In [52]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))
     




ACCIONA ofrece servicios en las siguientes áreas:

1. Energía: gestión integral del uso de la energía, enfocándose en la sostenibilidad y la digitalización, y diseño de soluciones para reducir y optimizar el consumo energético.
2. Transporte: incluye asistencia y transporte sanitario urgente y no urgente.
3. Agua: tratamiento de agua y gestión hídrica.
4. Sociales: planes específicos de formación continua, gestión de calidad del servicio, contingencias, emergencias y autoprotección para el personal.
5. Ciudades: soluciones de limpieza y plataformas innovadoras para la gestión de la logística interna.
6. Inmobiliaria: proyectos destacados en el sector salud, como el Hospital Infanta Sofía y Hospital de Vigo.
7. Financieras: no se especifica en el texto proporcionado.

Además, ACCIONA ofrece servicios de salud y bienestar a sus empleados, promoviendo la vigilancia de la salud de los trabajadores, campañas de prevención y detección precoz de enfermedades, acuerdos con compañías de salud privadas, servicios de nutricionista y acuerdos con gimnasios en condiciones ventajosas para los trabajadores.

la respuesta a esto es muy convincente y se puede mejorar dependiendo de la calidad de los datos, en caso de que los datos no esten disponibles el asistente simplemente no los va aindexar, aun asi vemos que se confunde un poco podriamos mejorar la preguntas cambiandola por los servicios para sus clientes, y los servicios para sus empleados

En este ejemplo, el LLMChain se utiliza para generar una respuesta pasando la entrada del usuario y formateándola utilizando la plantilla de la consulta. Luego se obtiene la respuesta de la instancia LLM (en este caso, OpenAI), y el texto generado se puede acceder desde el diccionario de respuesta.

Veamos esto comparado con una consulta no aumentada...

In [53]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

No tengo información sobre Acciona proporcionada. Por favor, proporcione más detalles o información para ayudarlo a responder su pregunta.

In [54]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))
     

Acciona es una empresa multinacional española que se especializa en el desarrollo y gestión de infraestructuras, energías renovables, agua y servicios. Algunos de los servicios principales que ofrece Acciona incluyen:

1. Energías Renovables: Desarrollo, construcción, operación y mantenimiento de parques eólicos, solares fotovoltaicos, termosolares, hidroeléctricos y plantas de biomasa.

2. Infraestructuras: Diseño, construcción, financiamiento y mantenimiento de infraestructuras públicas y privadas, como carreteras, ferrocarriles, hospitales, puertos, instalaciones deportivas y culturales.

3. Gestión del ciclo integral del agua: Construcción y operación de instalaciones de tratamiento de agua potable y aguas residuales, desalación de agua de mar, mantenimiento de redes de distribución y saneamiento, y gestión eficiente de los recursos hídricos.

4. Servicios Medioambientales: Gestión integral de residuos, incluyendo la recogida, transporte, tratamiento y valorización, y servicios de limpieza urbana.

5. Servicios Industriales: Actividades de ingeniería, construcción y mantenimiento en los sectores de energía, petroquímica, farmacéutica, aeronáutico y automotriz.

6. Movilidad y logística: Desarrollo de soluciones para la movilidad urbana e interurbana en transporte público y privado, además de servicios logísticos.

7. Concesiones y financiamiento de proyectos: Participación en proyectos de infraestructuras y energías renovables a través de concesiones, acuerdos de financiamiento y gestión de activos.

8. Innovación y desarrollo tecnológico: Investigación y desarrollo de tecnologías y soluciones sostenibles en energías renovables, infraestructuras y gestión del agua.